#  Rclone + Sleap Installation on AppStream (Linux Platform)

This report contains instructions to install rclone, a service to mount S3 buckets on the local file system, as well as conda, a package manager to install softwares such as Sleap. Users will be able to lancuh Sleap and analyze videos from S3 buckets directly. This report is for Amazon Linux 2 Images only.

- [Linux User Instruction](#Linux-User-Instruction)
- [Linux Build Image Instruction](#Linux-Build-Image-Instruction)


## Linux User Instruction

Sleap will be the default conda environment in the resulting image, so the users don't have to activate conda environments. 

### Launch Sleap GUI

    sleap-label
    
### Mount and unmount s3 bucket
    
    # mount s3 bucket
    mkdir MOUNT_PATH
    rclone mount s3:aind-appstream-data-dev-temporary MOUNT_PATH --daemon
    
    # unmount s3 bucket
    fusermount u MOUNT_PATH


## Linux Build Image Instruction

In the AppStream console Image tab, select Launch Image Builder. Filter for `Platform=Amazon Linux 2` and select the latest image. Make sure `Instance Family=Graphics G4` if you want NVIDIA GPU support. Enable default Internet access for simplicity. If applicable, attach an IAM role with access to S3 buckets that can be assumed be Appstream services. 


You would need to attach IAM role which has S3 bucket access (i.e. `appstream_data_bucket_role`) to an image builder or fleet in order for AppStream to create the AWS profile `appstream_machine_role`. The image builder instance or fleet instances will then be able to to access the S3 bucket (i.e. `aind-appstream-data-dev-0tvf3cjlng0m`) either through AWS CLI or through Rclone.

### Linux Install Sleap via Conda

It is important that when you install conda packages, you use `sudo /opt/miniconda3/bin/conda` instead of the usual `conda` command. The former will create the environment / packages under the `/opt/miniconda3/envs` folder which is available to all users, but the later will by defualt create them under the Image Builder Admin home directory, which is only visible to the Imgae Builder Admin.

    # install miniconda:
    wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    sudo bash Miniconda3-latest-Linux-x86_64.sh -p /opt/miniconda3 -b 
    sudo sh -c "echo $'export PATH="/opt/miniconda3/bin:$PATH"\nexport CONDA_ENVS_PATH="/opt/miniconda3/envs"' >> /etc/profile.d/conda.sh"
    source /etc/profile.d/conda.sh # or open a new terminal window
    
    # install sleap environment
    sudo /opt/miniconda3/bin/conda create -n sleap -c sleap -c nvidia -c conda-forge sleap=1.3.0 -y
    
Next you may run the following to set up conda initiation and default sleap environment for all streaming users. Alternatively, instructions in [Linux Default Settings](#Linux-Default-Settings-(Skipped)) accomplishes the same goal.

    # configure conda:
    conda init
    echo "conda activate sleap" >> .bashrc
    sudo cp .bashrc /etc/skel/
    
    # configure sleap:
    source .bashrc
    sleap-label
    sudo cp .sleap .keras .nv .config/matplotlib /etc/skel -r --parents

### Linux install Rclone

To specify a profile name while using rclone, run rclone commands with the flag `--s3-profile appstream_machine_role`, or set the environment variable `AWS_PROFILE`, or specify in the config file (as shown below). 

    # Installing dependencies
    sudo -v ; curl https://rclone.org/install.sh | sudo bash
    sudo amazon-linux-extras install epel -y && sudo yum install fuse3 -y

    # run rclone to create empty ~/.config/rclone/ 
    rclone 
    
    # Create ~/.config/rclone/rclone.conf with contents:
    [s3]
    type = s3
    provider = AWS
    env_auth = true
    region = us-west-2
    location_constraint = us-west-2
    profile = appstream_machine_role
    
    # Check rclone works
    rclone ls s3:aind-appstream-data-dev-temporary
    
    # In .bashrc: mount bucket with rclone as daemon process (i.e. in background)
    export MOUNT_PATH=${HOME}/s3-mount
    rclone mount s3:aind-appstream-data-dev-temporary $MOUNT_PATH --daemon
    
    # Move config file to image builder skeleton
    sudo cp .config/rclone/rclone.conf /etc/skel/ --parents
    sudo cp .bashrc /etc/skel/
    sudo mkdir /etc/skel/s3-mount
    
    # To unmount bucket
    usermount -u $MOUNT_PATH

### Linux Default Settings (You may skip this)

Create a Template User and set up default configurations, then copy them to `/etc/skel`. For this demo, we run `conda init` as a Template User and make sleap the default conda environment, so that streaming users can use the sleap software immediately. 

Initiating conda would effectivly modify the `.bashrc` file (so theoretically you could also directly modify the `/etc/skel/.bashrc` file without creating a Template User). You may also open the Sleap GUI while in TemplateUser so that the configurations file (i.e. `sleap/1.3.0/preferences.yaml`) would be created for the first time, and can get captured during optimization, so streaming users can open the GUI much faster. 

    sudo useradd -m TemplateUser
    echo -e '<password>\n<password>\n' | sudo passwd TemplateUser # fill in <password> with your choice of password
    su TemplateUser
    cd $HOME
    
    # initialize conda and set sleap as default env
    conda init
    echo "conda activate sleap" >> .bashrc
    
    # create sleap config files
    source .bashrc
    sleap-label
    
    exit
    sudo cp -r -f /home/TemplateUser/. /etc/skel
    sudo killall -u TemplateUser
    sudo userdel -r TemplateUser
    
You may also set environment variables as a Template User, but for this demo it is being set using the Image Assistant console. 
    
    # export AWS_PROFILE="appstream_machine_role" 

![llinux-var-console.png](attachment:llinux-var-console.png)


### Linux Test Settings

You may create another Test User to check that the configuration files are created correctly in the user's home directory. You should notice that the base conda environment is automatically activated, indicating that conda has already been initialized. If you launch the sleap GUI once again, it should open up with a much shorter wait time. 

    sudo useradd -m TestUser
    echo -e 'temppassword\ntemppassword\n' | sudo passwd TestUser
    su TestUser
    
    # Test sleap GUI launch time
    conda activate sleap
    time sleap-label

### Linux Create Image

Use the Image Assistance and add Terminal as app. 

![linux-terminal-config.png](attachment:linux-terminal-config.png)